<a href="https://colab.research.google.com/github/dipankerz/RVC_TPU/blob/main/RVC_TPU_0.0.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Google Colab banned the Gradio UI for free users, but you can still train models and infer locally or using a hugginface space.

In [ ]:
# Install and Setup Project on Google Drive
# This Notebook is based on another found in: https://github.com/ardha27/AI-Song-Cover-RVC
# Visit the repo to read more and support.

import os
from google.colab import drive
from IPython.display import clear_output

# Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')

# Ensure Google Drive is properly mounted
if not os.path.exists('/content/drive'):
    print("Your drive is not mounted. Creating a fake drive for demonstration purposes.")
    os.makedirs('/content/drive/MyDrive')

# Set up the source repository
source = "Rejekts"  # Replace with the desired source if different

# Download and unzip project files into Google Drive
print("Downloading and setting up project files...")
!wget https://huggingface.co/Rejekts/project/resolve/main/project-main.zip -O '/content/project-main.zip'
!unzip -n 'project-main.zip' -d /content/drive/MyDrive

# Navigate to the project directory and set up dependencies
print("Installing dependencies...")
!cd '/content/drive/MyDrive/project-main' && python download_files.py && pip install -r 'requirements-safe.txt'

# Install additional dependencies
!pip install pyngrok

# Cleanup unnecessary files and create necessary directories
print("Cleaning up unnecessary files...")
!rm /content/project-main.zip
!rm -r /content/sample_data
!mkdir -p /content/dataset

# Clear the output to reduce clutter
clear_output()

print("Setup complete. Project files are now ready in Google Drive.")


In [ ]:
#@title Install to Google Drive (for Resuming Training & Automatic Saving)
#@markdown <small> This Notebook is based on another found in: https://github.com/ardha27/AI-Song-Cover-RVC < Visit this repo to read more and support.
%cd /content
from google.colab import drive
drive.mount('/content/drive')
from IPython.display import clear_output
from ipywidgets import Button
import os
if not os.path.exists('/content/drive'):
    print("Your drive is not mounted. Creating Fake Drive.")
    os.makedirs('/content/drive/MyDrive')
source = "Rejekts"
!wget https://huggingface.co/{source}/project/resolve/main/project-main.zip -O '/content/project-main.zip' && unzip -n 'project-main.zip' -d /content/drive/MyDrive
!cd '/content/drive/MyDrive/project-main' && python download_files.py && pip install -r 'requirements-safe.txt'
!pip install pyngrok
!rm /content/project-main.zip
!rm -r /content/sample_data
!mkdir -p /content/dataset


In [ ]:
#@title 1.Preprocess Data
%cd /content/drive/MyDrive/project-main
model_name = 'NeilModel' #@param {type:"string"}
#@markdown <small> Enter the path to your dataset folder (a folder with audios of the vocals you will train on), or if you want just upload the audios using the File Manager into the 'dataset' folder.
dataset_folder = '/content/dataset' #@param {type:"string"}
while len(os.listdir(dataset_folder)) < 1:
    input("Your dataset folder is empty.")
!mkdir -p ./logs/{model_name}
with open(f'./logs/{model_name}/preprocess.log','w') as f:
    print("Starting...")
!python infer/modules/train/preprocess.py {dataset_folder} 40000 2 ./logs/{model_name} False 3.0 > /dev/null 2>&1
with open(f'./logs/{model_name}/preprocess.log','r') as f:
    if 'end preprocess' in f.read():
        clear_output()
        display(Button(description="\u2714 Success", button_style="success"))
    else:
        print("Error preprocessing data... Make sure your dataset folder is correct.")

In [ ]:
!pip install numpy==1.23.5

In [ ]:
#@title 2.Extract Features
f0method = "rmvpe_gpu" # @param ["pm", "harvest", "rmvpe", "rmvpe_gpu"]
%cd /content/drive/MyDrive/project-main
with open(f'./logs/{model_name}/extract_f0_feature.log','w') as f:
    print("Starting...")
if f0method != "rmvpe_gpu":
    !python infer/modules/train/extract/extract_f0_print.py ./logs/{model_name} 2 {f0method}
else:
    !python infer/modules/train/extract/extract_f0_rmvpe.py 1 0 0 ./logs/{model_name} True
!python infer/modules/train/extract_feature_print.py cuda:0 1 0 0 ./logs/{model_name} v2
with open(f'./logs/{model_name}/extract_f0_feature.log','r') as f:
    if 'all-feature-done' in f.read():
        clear_output()
        display(Button(description="\u2714 Success", button_style="success"))
    else:
        print("Error preprocessing data... Make sure your data was preprocessed.")

In [ ]:
#@title 3.Train Index
import numpy as np
import faiss
%cd /content/drive/MyDrive/project-main
def train_index(exp_dir1, version19):
    exp_dir = "logs/%s" % (exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if not os.path.exists(feature_dir):
        return "请先进行特征提取!"
    listdir_res = list(os.listdir(feature_dir))
    if len(listdir_res) == 0:
        return "请先进行特征提取！"
    infos = []
    npys = []
    for name in sorted(listdir_res):
        phone = np.load("%s/%s" % (feature_dir, name))
        npys.append(phone)
    big_npy = np.concatenate(npys, 0)
    big_npy_idx = np.arange(big_npy.shape[0])
    np.random.shuffle(big_npy_idx)
    big_npy = big_npy[big_npy_idx]
    if big_npy.shape[0] > 2e5:
        infos.append("Trying doing kmeans %s shape to 10k centers." % big_npy.shape[0])
        yield "\n".join(infos)
        try:
            big_npy = (
                MiniBatchKMeans(
                    n_clusters=10000,
                    verbose=True,
                    batch_size=256 * config.n_cpu,
                    compute_labels=False,
                    init="random",
                )
                .fit(big_npy)
                .cluster_centers_
            )
        except:
            info = traceback.format_exc()
            logger.info(info)
            infos.append(info)
            yield "\n".join(infos)

    np.save("%s/total_fea.npy" % exp_dir, big_npy)
    n_ivf = min(int(16 * np.sqrt(big_npy.shape[0])), big_npy.shape[0] // 39)
    infos.append("%s,%s" % (big_npy.shape, n_ivf))
    yield "\n".join(infos)
    index = faiss.index_factory(256 if version19 == "v1" else 768, "IVF%s,Flat" % n_ivf)
    infos.append("training")
    yield "\n".join(infos)
    index_ivf = faiss.extract_index_ivf(index)  #
    index_ivf.nprobe = 1
    index.train(big_npy)
    faiss.write_index(
        index,
        "%s/trained_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )

    infos.append("adding")
    yield "\n".join(infos)
    batch_size_add = 8192
    for i in range(0, big_npy.shape[0], batch_size_add):
        index.add(big_npy[i : i + batch_size_add])
    faiss.write_index(
        index,
        "%s/added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )
    infos.append(
        "成功构建索引，added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (n_ivf, index_ivf.nprobe, exp_dir1, version19)
    )

training_log = train_index(model_name, 'v2')

for line in training_log:
    print(line)
    if 'adding' in line:
        clear_output()
        display(Button(description="\u2714 Success", button_style="success"))

In [ ]:
# Train Model Script
# Import necessary modules
from random import shuffle
import json
import os
import pathlib
from subprocess import Popen, PIPE, STDOUT
from pyngrok import ngrok

# Global Variables
now_dir = os.getcwd()

# User Inputs
ngrok_authtoken = "2Z6r4I6fad3L2Chu7hSb5f9qdcm_4wTKeMnUKrguTkNg48TD3"  # NGROK authtoken
model_name = 'NeilModel'  # Model name
save_frequency = 50  # Frequency to save the model
epochs = 250  # Number of training epochs
cache = False  # Whether to cache datasets

# Configure ngrok
os.system(f"ngrok config add-authtoken {ngrok_authtoken}")

def setup_directories(exp_dir, gt_wavs_dir, feature_dir, f0_dir=None, f0nsf_dir=None):
    """Ensure necessary directories exist."""
    os.makedirs(exp_dir, exist_ok=True)
    os.makedirs(gt_wavs_dir, exist_ok=True)
    os.makedirs(feature_dir, exist_ok=True)
    if f0_dir:
        os.makedirs(f0_dir, exist_ok=True)
    if f0nsf_dir:
        os.makedirs(f0nsf_dir, exist_ok=True)

def generate_filelist(gt_wavs_dir, feature_dir, spk_id, f0_dir=None, f0nsf_dir=None, if_f0=False, version="v2"):
    """Generate the filelist for training."""
    names = set(name.split(".")[0] for name in os.listdir(gt_wavs_dir)) & \
            set(name.split(".")[0] for name in os.listdir(feature_dir))

    if if_f0:
        names &= set(name.split(".")[0] for name in os.listdir(f0_dir))
        names &= set(name.split(".")[0] for name in os.listdir(f0nsf_dir))

    fea_dim = 256 if version == "v1" else 768
    filelist = []

    for name in names:
        if if_f0:
            filelist.append(
                f"{gt_wavs_dir}/{name}.wav|{feature_dir}/{name}.npy|{f0_dir}/{name}.wav.npy|"
                f"{f0nsf_dir}/{name}.wav.npy|{spk_id}"
            )
        else:
            filelist.append(f"{gt_wavs_dir}/{name}.wav|{feature_dir}/{name}.npy|{spk_id}")

    # Adding muted entries
    for _ in range(2):
        if if_f0:
            filelist.append(
                f"{now_dir}/logs/mute/0_gt_wavs/mute40k.wav|{now_dir}/logs/mute/3_feature{fea_dim}/mute.npy|"
                f"{now_dir}/logs/mute/2a_f0/mute.wav.npy|{now_dir}/logs/mute/2b-f0nsf/mute.wav.npy|{spk_id}"
            )
        else:
            filelist.append(
                f"{now_dir}/logs/mute/0_gt_wavs/mute40k.wav|{now_dir}/logs/mute/3_feature{fea_dim}/mute.npy|{spk_id}"
            )

    shuffle(filelist)
    return filelist

def click_train(exp_dir, model_config, filelist):
    """Execute the training process."""
    # Save the filelist
    filelist_path = os.path.join(exp_dir, "filelist.txt")
    with open(filelist_path, "w") as f:
        f.write("\n".join(filelist))

    print("Filelist written successfully.")

    # Training command
    cmd = (
        f'python infer/modules/train/train.py -e "{model_config["exp_dir"]}" -sr {model_config["sample_rate"]} '
        f'-f0 {1 if model_config["if_f0"] else 0} -bs {model_config["batch_size"]} -g {model_config["gpus"]} '
        f'-te {model_config["total_epochs"]} -se {model_config["save_epochs"]} '
        f'{f"-pg {model_config["pretrained_G"]}" if model_config["pretrained_G"] else ""} '
        f'{f"-pd {model_config["pretrained_D"]}" if model_config["pretrained_D"] else ""} '
        f'-l {1 if model_config["save_latest"] else 0} -c {1 if model_config["cache_gpu"] else 0} '
        f'-sw {1 if model_config["save_weights"] else 0} -v {model_config["version"]}'
    )

    # Run the command
    with Popen(cmd, shell=True, cwd=now_dir, stdout=PIPE, stderr=STDOUT, bufsize=1, universal_newlines=True) as proc:
        for line in proc.stdout:
            print(line.strip())
        proc.wait()

    print("Training completed.")
    return "Training finished. Check logs for details."

# Tensorboard setup
from IPython import get_ipython
get_ipython().run_line_magic("load_ext", "tensorboard")
get_ipython().run_line_magic("tensorboard", "--logdir ./logs --port=8888")

# Start ngrok for tensorboard
print("Tensorboard NGROK URL:", end="")
try:
    ngrok_url = ngrok.connect(8888)
    print(ngrok_url)
    model_config = {
        "exp_dir": model_name,
        "sample_rate": "40k",
        "if_f0": True,
        "batch_size": 7,
        "save_epochs": save_frequency,
        "total_epochs": epochs,
        "pretrained_G": "assets/pretrained_v2/f0G40k.pth",
        "pretrained_D": "assets/pretrained_v2/f0D40k.pth",
        "gpus": 0,
        "cache_gpu": cache,
        "save_latest": True,
        "save_weights": True,
        "version": "v2"
    }
    gt_wavs_dir = f"{now_dir}/logs/{model_name}/0_gt_wavs"
    feature_dir = f"{now_dir}/logs/{model_name}/3_feature768"
    setup_directories(model_name, gt_wavs_dir, feature_dir)
    filelist = generate_filelist(gt_wavs_dir, feature_dir, 0, if_f0=True)
    training_log = click_train(model_name, model_config, filelist)
    print(training_log)
except Exception as e:
    print("Error:", e)
    ngrok.kill()


In [ ]:
#@title OR Open the GUI (Banned for free Colab Notebooks)
if not 'installed' in locals():
    %cd /content
    from google.colab import drive
    drive.mount('/content/drive')
    from IPython.display import clear_output
    from ipywidgets import Button
    import os
    if not os.path.exists('/content/drive'):
        print("Your drive is not mounted. Creating Fake Drive.")
        os.makedirs('/content/drive/MyDrive')
    if not os.path.exists('/content/drive/MyDrive/project-main'):
        !wget https://huggingface.co/Rejekts/project/resolve/main/project-main.zip -O '/content/project-main.zip' && unzip 'project-main.zip' -d /content/drive/MyDrive
    !cd '/content/drive/MyDrive/project-main' && python download_files.py && pip install -r 'requirements.txt'
    !rm /content/project-main.zip
    !rm -r /content/sample_data
    !mkdir -p /content/dataset
    clear_output()
    Button(description="\u2714 Success", button_style="success")
tensorboard = True #@param {type:"boolean"}
if tensorboard:
    %load_ext tensorboard
    %tensorboard --logdir ./logs
%cd /content/drive/MyDrive/project-main
!python app.py --colab